## Goal:

This example focuses on using lidarSuit to retrieve wind speed and direction profiles from the observations collected by the WindCube using the 6-beam scan strategy. 

## Steps:

1) Read the WindCube's data
2) Merge data from one hour of observations
3) Re-structure the data for using the wind retrieval
4) Retrieve wind profiles using an FFT-based method
5) Saving as NetCDF

In [1]:
import glob as gb
import matplotlib.pyplot as plt

import lidarSuit as lst

### Step 1 and 2: reading and merging the 6-beam data

In [2]:
# a 6-beam sample data is available at the path below
data_path = '../../sampleData/wind_and_aerosols_data/12-00/*.nc'
file_list = sorted(gb.glob(data_path))
mergedDS = lst.dataOperations(file_list).mergedData

Below you can see all the variables available on the original WindCube's data. Not all of them are needed for retrieving the wind profiles. Note that the variables from the vertical observations are kept separate from the slanted observation. 

In [3]:
# Uncomment the line below

# mergedDS

First inpressions from the vertical and slanted observations

In [4]:
# Uncomment the lines below

# mergedDS.radial_wind_speed.plot(x='time', cmap='Spectral',vmin=-1, vmax=1)
# plt.show()

# mergedDS.radial_wind_speed90.plot(x='time', cmap='Spectral', vmin=-1, vmax=1)
# plt.show()

### Step 3: re-structuring the data

In [5]:
restruct_data = lst.getRestructuredData(mergedDS)

The re-structured object contains all the information needed to retrieve the wind profiles. You can have a look again at the slanted and vertical observations.

In [6]:
# Radial observation at an elevation of 75 degrees and an azimuth of 216 degrees
# Uncomment the lines below

# restruct_data.dataTransf.isel(azm=3).plot(x='time',cmap='Spectral',vmin=-1, vmax=1)
# plt.show()

# Observations from the zenith pointing measurements
# Uncomment the lines below

# restruct_data.dataTransf90.plot(x='time', cmap='Spectral', vmin=-1, vmax=1)
# plt.show()

### Step 4: retrieving wind profiles

In [7]:
wind_obj = lst.retrieveWind(restruct_data)

/Users/jdiasneto/.local/lib/python3.8/site-packages/xrft/xrft.py:338: FutureWarning: Flags true_phase and true_amplitude will be set to True in future versions of xrft.dft to preserve the theoretical phasing and amplitude of Fourier Transform. Consider using xrft.fft to ensure future compatibility with numpy.fft like behavior and to deactivate this warning.
  warnings.warn(msg, FutureWarning)


After retrieving the wind profiles, the dataset can be accessed by calling the windProp attribute. You can also visualize the retrieved wind speed and direction pofiles. Have a look below.

In [8]:
# Uncomment the line below

# wind_obj.windProp

In [9]:
# Uncomment the lines below to see the plots

# wind_obj.windProp.horizontal_wind_speed.plot(y='range', cmap='turbo')
# plt.show()

# wind_obj.windProp.horizontal_wind_direction.plot(y='range', cmap='hsv')
# plt.show()

### Step 5: Saving as NetCDF

In [10]:
# Uncomment the line below

# wind_obj.windProp.to_netcdf('6_beam_example.nc')